In [105]:
import numpy as np
import pandas as pd
import string
import json

from numpy import *
from pylab import *
from math import *
import re
import time
import codecs

In [144]:
f = open('imdb_data.json',)
data = json.load(f)
   
file = codecs.open('stopwords.dic','r','utf-8')
stopwords = [document.strip() for document in file] 
file.close()

array = []
for movie in data:
    array.append(movie['movie_storyLines'])
f.close()

movID = []
for movie in data:
    movID.append(movie['movie_id'])
  
print(movID[0])
print(array[500])


tt2488496
["It's 1969 at a strict English girls' school where charismatic Abbie and intense and troubled Lydia are best friends. After a tragedy occurs at the school, a mysterious fainting epidemic breaks out threatening the stability of all involved.", "At a strict English girls' boarding school in 1969, charismatic Abbie and intense and troubled Lydia are best friends. After a tragedy occurs at the school, a mysterious fainting epidemic breaks out that threatens the stability of all involved."]


In [145]:
#documents:电影相关文章array
#K:topics
#topicWordsNum

def PLSA(docArr, topics, topicWords, stopwordsArr):
    documents = docArr
    K = topics
    topicWordsNum = topicWords
    stopwords = stopwordsArr


    N = len(documents)
    wordCount = {}
    wordCountPerDocument = [];
    punctuationRegex = '[,.;"?!#-_…()`|“”‘]+'


    for d in documents:
        words = d.split()
        wordCountCurrentDoc = {}
        for w in words:
            w = re.sub(punctuationRegex, '', w.lower())
            if len(w)<=1 or re.search('http', w) or re.search('[0-9]', w) or w in stopwords:
                continue
            if w in wordCount:
                wordCount[w] += 1
            else:
                wordCount[w] = 1
            if w in wordCountCurrentDoc:
                wordCountCurrentDoc[w] += 1
            else:
                wordCountCurrentDoc[w] = 1
        wordCountPerDocument.append(wordCountCurrentDoc);

    dictionary = {}
    dictionaryReverse = {}

    index = 0;
    for word in wordCount.keys():
        if wordCount[word] > 1:
            dictionary[word] = index;
            dictionaryReverse[index] = word;
            index += 1;

    M = len(dictionary)  

    X = zeros([N, M], int8)

    for word in dictionary.keys():
        j = dictionary[word]
        for i in range(0, N):
            if word in wordCountPerDocument[i]:
                X[i, j] = wordCountPerDocument[i][word];

    lamda = random([N, K])
    for i in range(0, N):
        normalization = sum(lamda[i, :])
        for j in range(0, K):
            lamda[i, j] /= normalization;

    theta = random([K, M])
    for i in range(0, K):
        normalization = sum(theta[i, :])
        for j in range(0, M):
            theta[i, j] /= normalization;

    p = zeros([N, M, K])

    for i in range(0, 200):
        #EStep()
        for i in range(0, N):
            for j in range(0, M):
                denominator = 0;
                for k in range(0, K):
                    p[i, j, k] = theta[k, j] * lamda[i, k];
                    denominator += p[i, j, k];
                if denominator == 0:
                    for k in range(0, K):
                        p[i, j, k] = 0;
                else:
                    for k in range(0, K):
                        p[i, j, k] /= denominator;
        #MStep()
        for k in range(0, K):
            denominator = 0
            for j in range(0, M):
                theta[k, j] = 0
                for i in range(0, N):
                    theta[k, j] += X[i, j] * p[i, j, k]
                denominator += theta[k, j]
            if denominator == 0:
                for j in range(0, M):
                    theta[k, j] = 1.0 / M
            else:
                for j in range(0, M):
                    theta[k, j] /= denominator
        for i in range(0, N):
            for k in range(0, K):
                lamda[i, k] = 0
                denominator = 0
                for j in range(0, M):
                    lamda[i, k] += X[i, j] * p[i, j, k]
                    denominator += X[i, j];
                if denominator == 0:
                    lamda[i, k] = 1.0 / K
                else:
                    lamda[i, k] /= denominator

    topicwords = []
    for i in range(0, K):
        topicword = []
        ids = theta[i, :].argsort()
        for j in ids:
            topicword.insert(0, dictionaryReverse[j])
        topicwords.append(topicword[0:min(topicWordsNum, len(topicword))])
    return topicwords

In [146]:
a = PLSA(array[0], 1, 10, stopwords)

len(array)

511

In [147]:
movIDandTopics = []

assert (len(array) == len(movID)), "movID != array!!!!"
for i in range(len(movID)):
    movIDandTopics.append((movID[i],PLSA(array[i], 1, 10, stopwords)))
    

In [150]:
movIDandTopics


[('tt2488496',
  [['droid',
    'resistance',
    'rey',
    'scavenger',
    'luke',
    'empire',
    'galaxy',
    'skywalker',
    'jedi',
    'stormtrooper']]),
 ('tt1408101',
  [['crew',
    'kirk',
    'enterprise',
    'starfleet',
    'takes',
    'mission',
    'uss',
    'captain',
    'manhunt',
    'unstoppable']]),
 ('tt0121766',
  [['anakin',
    'jedi',
    'master',
    'obi',
    'chancellor',
    'wan',
    'skywalker',
    'palps',
    'darth',
    'council']]),
 ('tt1517451',
  [['jackson',
    'ally',
    'bobby',
    'career',
    'singer',
    'invites',
    'performance',
    'manager',
    'music',
    'relationship']]),
 ('tt0120915',
  [['jedi',
    'naboo',
    'queen',
    'planet',
    'boy',
    'jar',
    'quigon',
    'federation',
    'trade',
    'jinn']]),
 ('tt4587656',
  [['donkey',
    'bo',
    'named',
    'star',
    'mary',
    'months',
    'christmas',
    'dove',
    'dave',
    'joseph']]),
 ('tt2660888',
  [['kirk',
    'federation',
   